# Analyse de Données - Développeur
## TP4 : Classification Supervisée 
## 0) Tableau de Données de l'Exercice 1 du TD4 

In [17]:
import numpy as np
import pandas as pd
df = pd.read_excel("TD4-Ex1.xlsx")
print(df)
X = df.values[:,:2]
Y = df.values[:,2]

    X1  X2  Y
0    0   2  1
1    2   0  1
2    2   6  1
3    4   4  1
4    2   4  2
5    6   7  2
6   10   4  2
7    4   0  3
8    4   2  3
9    6   0  3
10   6   2  3


## 1) Nearest Prototype Classifier
### 1.1) Disposez-vous de toutes les fonctions utiles au codage de cette règle de classification supervisée ? Importez-les. 

In [25]:
def dist(x,y,dname='euclidean'):
    d = np.abs(x-y)
    if dname == 'manhattan' or dname == 'cityblock':
        return sum(d)
    elif dname in ['chebyshev', 'chebychev']:
        return max(d)
    elif dname in ["cosinus", "cosine", "cos"]:
        return 1 - x.dot(y)/np.sqrt(x.dot(x)*y.dot(y))
    else:
        return np.sqrt(sum(d**2))

def clustering(data,centroids,group_names=None,dname='eulidean'):
    """Group data by clusters, given some centroids."""
    # default to 1, 2, 3, …, n
    if group_names is None:
        group_names = [i+1 for i in range(len(centroids))]

    distances = []
    groups = []
    for d in data:
        t = []
        for c in centroids:
            # On détermine la distance au centroid
            t.append(dist(d,c,dname))
        # On détermine le groupe pour cet individu
        group = t.index(min(t))
        groups.append(group_names[group])
        distances.append(t)
    # Argmin peut faire des trucs cools, genre retourner l'index de la valeur min
    return np.array(distances), groups

def prototyping(data, partition):
    clust_id = np.unique(partition)
    centroids = []
    for id in clust_id:
        elms = data[partition == id]
        centroids.append(elms.mean(axis=0))
    return np.array(centroids)

def partitionmatrix(data_clusters):
    data_clusters = np.array(data_clusters)
    n = data_clusters.shape[0]
    clust_id = np.unique(data_clusters)
    clust_nb = len(clust_id)
    data_partition = np.zeros((n,clust_nb))
    for k in range(n):
        data_partition[k,clust_id==data_clusters[k]] = 1
    return data_partition

### 1.2) Ecrivez une fonction *NPClassifier* permettant de prédire, pour tous les points d'un tableau *test*, leur prédiction *test_labels* avec la règle du Plus Proche Prototype d'un ensemble d'apprentissage (tableau *learn*, indicatice de groupe *learn_labels*) au sens d'une distance *dname* (euclidienne par défaut). 
### Vous testerez sur un tableau *Xtest* comprenant les points $x$, $y$ et $z$ du TD.

In [61]:
Xtest = np.array([[3,2],[10,2],[5,3]])
def NPClassifier(test, learn, learn_labels,dname='euclidean'):
    prototypes = prototyping(learn,learn_labels)
    test_labels = clustering(test,prototypes,None,dname)[1]
    return test_labels

Ytest = NPClassifier(Xtest,X,Y)
Ypred = NPClassifier(X,X,Y)
print(Ytest)
Ypred

[1, 2, 3]


[1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3]

### 1.3) Reclassez les données d'apprentissage. Tous les points sont-ils bien reclassés ? 

### 1.4) Trouvez un moyen 
#### 1.4.1) simple de calculer la matrice de confusion (table de contingence croisant les *learn_labels* et les *pred_labels* d'un ensemble d'apprentissage (tableau *learn*, indicatice de groupe *learn_labels*)


In [56]:
learn_part = partitionmatrix(Y)
test_part = partitionmatrix(Ypred)
confusion =  np.dot(learn_part.T,test_part)
confusion

array([[4., 0., 0.],
       [1., 2., 0.],
       [0., 0., 4.]])

#### 1.4.2) en déduire la *Classification Accuracy*

In [57]:
def classif_acc(actual,predicted):
    learn_part = partitionmatrix(actual)
    pred_part = partitionmatrix(predicted)
    confusion =  np.dot(learn_part.T,pred_part)
    accuracy = np.diag(confusion).sum() / confusion.sum()
    return accuracy, confusion

CA, conf = classif_acc(Y,Ypred)

print(conf)
print(CA)

[[4. 0. 0.]
 [1. 2. 0.]
 [0. 0. 4.]]
0.9090909090909091


In [58]:
a = [3,1,1,2,3,3,2,2,3,3,1,1,1,1]
p = [3,1,2,2,1,3,2,3,2,3,1,1,1,2]
classif_acc(a,p)

(0.6428571428571429, array([[4., 2., 0.],
        [0., 2., 1.],
        [1., 1., 3.]]))

## 2) Nearest Neighbors Classifier
### 2.1) Ecrivez une fonction *NNClassifier* permettant de prédire, pour tous les points d'un tableau *test*, leur prédiction *test_labels* avec la règle des K Plus Proches Voisins d'un ensemble d'apprentissage (tableau *learn*, indicatice de groupe *learn_labels*) au sens d'une distance *dname* (euclidienne par défaut). 
### Pour déterminer le groupe majoritairement représenté parmi les *K* voisins, utilisez la fonction *histogram* de *numpy*.
### Vous testerez sur le tableau *Xtest* comprenant les points $x$, $y$ et $z$ du TD, avec *K=5*.

In [88]:
Xtest = np.array([[3,2],[10,2],[5,3]])
def NNClassifier(test, learn, learn_labels, K=1, dname='euclidean'):
    test_nb = test.shape[0]
    learn_nb = learn.shape[0]
    labels_id = np.unique(learn_labels)
    labels_nb = len(labels_id)
    dist2learn = np.zeros((test_nb,learn_nb))
    votes = np.zeros((test_nb,labels_nb))
    
    for i in range(test_nb):
        for k in range(learn_nb):
            dist2learn[i,k] = dist(test[i,:],learn[k,:],dname)
            # indexes des K-NN
            knn_index = np.argsort(dist2learn[i,:])[:K]
            knn_labels = learn_labels[knn_index]
            for j in range(labels_nb):
                votes[i,j] = len(np.argwhere(knn_labels == labels_id[j]))
   
    return knn_index, knn_labels, votes

NNClassifier(Xtest,X,Y,5)

(array([ 3,  8, 10,  4,  7]), array([1, 3, 3, 2, 3]), array([[2., 1., 2.],
        [1., 1., 3.],
        [1., 1., 3.]]))

### 2.2) Peut-on reclasser les données d'apprentisage ?

In [92]:
Xpred = NNClassifier(X,X,Y,5)[1]
CA, conf = classif_acc(Y,Ypred)
CA

0.9090909090909091

## 3) Données de *Machine Learning*

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_excel("../Data/MLlogiciel.xlsx")
T = df.values[5:,1:4]; 
T_labels = df.values[5:,:1]
Tbar = df.values[:5,1:4]; 
Tbar_labels= df.values[:5,:1]

### 3.1) En utilisant *NPCLassifier*, réalisez les prédictions pour les données du tableau *Tbar* à partir de l'ensemble d'apprentissage (*T,T_labels*) et calculez la *CA*.

### 3.2) Recommencez en échangeant les deux ensembles (*T,T_labels*) et (*Tbar,Tbar_labels*).

### 3.3) Reprenez les questions 3.1 et 3.2 avec *NNClassifier. Vous pourrez tester plusieurs valeurs de *K* et de *dname*.